In [1]:
import pickle
import pandas as pd
import numpy as np
import warnings
import re
warnings.filterwarnings('ignore')

In [2]:
# Intentar cargar el archivo CSV con diferentes configuraciones
df_locales = pd.read_csv("01_data\datos_locales_ampliado.csv", delimiter=";", header=0)
df_locales

,local,direccion,tipo_local,valoracion,numero_reseñas,enlace_web,puntuacion_entradas_com,numero_entradas_com,puntuacion_facebook,numero_puntuacion_facebook
0,Teatro Príncipe Gran Vía,"C. de las Tres Cruces, 8, 28013 Madrid",Teatro en Madrid,"4,2",2.601 reseñas de Google,http://www.teatroprincipegranvia.es/,"3,7/5",823 reseñas,"4,3/5",89 votos
1,El Golfo Comedy Club,"C. de las Huertas, 57, 28014 Madrid",Club de comedia,"4,7",1.082 reseñas de Google,https://elgolfocomedy.com/,"8,9/10",· 1.660 votos,NaN,NaN
2,Teatro Arlequín Gran Vía,"C. de San Bernardo, 5, 28013 Madrid",Teatro en Madrid,"4,2",5.264 reseñas de Google,http://teatroarlequingranvia.com/,"3,7/5",329 reseñas,"4,6/5",103 votos
3,Teatro Capitol Gran Vía,"C/ Gran Vía, 41, 28013 Madrid",Cine en Madrid,"4,2",4.902 reseñas de Google,http://www.capitolgranvia.com/,4/5,· 577 reseñas,NaN,NaN
4,Estación Malasaña,"C. del Pez, 16, 28004 Madrid",Bar,"4,5",344 reseñas de Google,https://www.estacionmalasana.es/,"3,9/5",· 23 votos,NaN,NaN
5,Teatro Alcázar,"C. de Alcalá, 20, 28014 Madrid",Teatro en Madrid,"4,3",6.730 reseñas de Google,https://gruposmedia.com/teatro-alcazar/,"3,9/5",· 1.554 reseñas,NaN,NaN
6,Cinesa Fuencarral,"Calle de Fuencarral, 136, 28010 Madrid",Cine en Madrid,"4,2",6.918 reseñas de Google,http://www.cinesa.es/Cines/Proyecciones,NaN,NaN,NaN,NaN
7,El Sombrerero Comedy Club,"C. de Trafalgar, 15, 28010 Madrid",Discoteca en Madrid,"4,5",4 reseñas de Google,NaN,NaN,NaN,NaN,NaN
8,Teatro de las Aguas,"C. de las Aguas, 8, 28005 Madrid",Teatro en Madrid,"4,2",1.848 reseñas de Google,http://www.teatrodelasaguas.com/,"9,5/10",902 votos,"4,7/5",116 votos
9,Abonavida,"C. de las Navas de Tolosa, 3, 28013 Madrid",€€Bar,"4,3",1.274 reseñas de Google,http://www.abonavida.com/,4/5,103 votos,"4,7/5",3 reseñas


In [3]:
# Miramos la información básica de cada una de las columnas del df
df_locales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   local                       52 non-null     object
 1   direccion                   48 non-null     object
 2   tipo_local                  47 non-null     object
 3   valoracion                  48 non-null     object
 4   numero_reseñas              46 non-null     object
 5   enlace_web                  44 non-null     object
 6   puntuacion_entradas_com     32 non-null     object
 7   numero_entradas_com         32 non-null     object
 8   puntuacion_facebook         15 non-null     object
 9   numero_puntuacion_facebook  15 non-null     object
dtypes: object(10)
memory usage: 4.2+ KB


In [4]:
# Contamos el porcentaje de valores nulos de cada columna
df_locales.isnull().sum()/df_locales.shape[0]

local                         0.000000
direccion                     0.076923
tipo_local                    0.096154
valoracion                    0.076923
numero_reseñas                0.115385
enlace_web                    0.153846
puntuacion_entradas_com       0.384615
numero_entradas_com           0.384615
puntuacion_facebook           0.711538
numero_puntuacion_facebook    0.711538
dtype: float64

# Limpiamos direcciones

In [5]:
# Filtramos las filas donde la columna df_locales[1] es nula
filas_nulas = df_locales[df_locales['direccion'].isnull()]
filas_nulas

,local,direccion,tipo_local,valoracion,numero_reseñas,enlace_web,puntuacion_entradas_com,numero_entradas_com,puntuacion_facebook,numero_puntuacion_facebook
25,MadBeerHall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,Así sí Malasaña,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,Tatemado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,Beer Point,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Veo que los locales estan mal nombrados en atrapalo y que como mucho tienen un show con poca información, asi que elimino estos locales y los shows asociados en el CSV de shows

In [6]:
df_locales = df_locales.dropna(subset=['direccion'])

In [7]:
# Hacemos una comprobación

df_locales.isnull().sum()/df_locales.shape[0]

local                         0.000000
direccion                     0.000000
tipo_local                    0.020833
valoracion                    0.000000
numero_reseñas                0.041667
enlace_web                    0.083333
puntuacion_entradas_com       0.333333
numero_entradas_com           0.333333
puntuacion_facebook           0.687500
numero_puntuacion_facebook    0.687500
dtype: float64

# Limpiamos tipo_local

In [8]:
# Miramos qué tenemos
df_locales['tipo_local'].unique()

array(['Teatro en Madrid', 'Club de comedia', 'Cine en Madrid', 'Bar',
       'Discoteca en Madrid', '€€Bar', '€€Pub irlandés', 'Teatro',
       'Espacio de coworking en Madrid', '€€Cafetería apta para perros',
       'Club de cómicos en Madrid', 'Café teatro en Madrid',
       '€Cervecería', 'Mago en Madrid', 'Bar restaurante',
       'Teatro en Leganés', 'Sala de conciertos en Madrid',
       'Hotel de 4 estrellas', 'Pub', 'Teatro en Alcalá de Henares',
       'Teatro en Aranjuez', nan, '€Café teatro, Rivas-Vaciamadrid',
       '€Pub', 'Hotel de 2 estrellas', '€€Cervecería artesanal',
       'Escuela de baile en Madrid'], dtype=object)

In [9]:
# Reemplazamos el símbolo '€' por una cadena vacía en la columna 'tipo_local'
df_locales['tipo_local'] = df_locales['tipo_local'].str.replace('€', '')
# Reemplazamos 'en Madrid' por una cadena vacía en la columna 'tipo_local'
df_locales['tipo_local'] = df_locales['tipo_local'].str.replace('en Madrid', '')
df_locales['tipo_local'].unique()

array(['Teatro ', 'Club de comedia', 'Cine ', 'Bar', 'Discoteca ',
       'Pub irlandés', 'Teatro', 'Espacio de coworking ',
       'Cafetería apta para perros', 'Club de cómicos ', 'Café teatro ',
       'Cervecería', 'Mago ', 'Bar restaurante', 'Teatro en Leganés',
       'Sala de conciertos ', 'Hotel de 4 estrellas', 'Pub',
       'Teatro en Alcalá de Henares', 'Teatro en Aranjuez', nan,
       'Café teatro, Rivas-Vaciamadrid', 'Hotel de 2 estrellas',
       'Cervecería artesanal', 'Escuela de baile '], dtype=object)

Para unificar valores y clasificarlos de una manera más concreta miramos el conteo de tipos de locales devuelto por google

In [10]:
# Obtenemos el conteo de ocurrencias de cada tipo de local
conteo_locales = df_locales.groupby('tipo_local')['local'].count().reset_index()

# Renombramos las columnas del DataFrame de conteo
conteo_locales.columns = ['tipo_local', 'conteo']
conteo_locales

,tipo_local,conteo
0,Bar,4
1,Bar restaurante,1
2,Cafetería apta para perros,1
3,Café teatro,1
4,"Café teatro, Rivas-Vaciamadrid",1
5,Cervecería,2
6,Cervecería artesanal,1
7,Cine,4
8,Club de comedia,1
9,Club de cómicos,1


In [11]:
# Unificamos los tipos de local
# Cambiamos los valores que contienen 'teatro' por 'teatro' ya sea en el 'tipo de local' como en el nombre del local
df_locales['tipo_local'] = np.where(df_locales['tipo_local'].str.contains('teatro', case=False), 'teatro', df_locales['tipo_local'])
df_locales['tipo_local'] = np.where(df_locales['local'].str.contains('teatro', case=False), 'teatro', df_locales['tipo_local'])
# Cambiamos los valores que contienen 'hotel' por 'hotel'
df_locales['tipo_local'] = np.where(df_locales['tipo_local'].str.contains('hotel', case=False), 'hotel', df_locales['tipo_local'])
# Cambiamos los valores que contienen 'bar' por 'bar'
df_locales['tipo_local'] = np.where(df_locales['tipo_local'].str.contains('bar', case=False), 'bar', df_locales['tipo_local'])
# Cambiamos los valores que contienen 'pub' por 'bar'
df_locales['tipo_local'] = np.where(df_locales['tipo_local'].str.contains('pub', case=False), 'bar', df_locales['tipo_local'])
# Cambiamos los valores que contienen 'Cervecería' por 'bar'
df_locales['tipo_local'] = np.where(df_locales['tipo_local'].str.contains('Cervecería', case=False), 'bar', df_locales['tipo_local'])
# Cambiamos los valores que contienen 'Cafetería' por 'bar'
df_locales['tipo_local'] = np.where(df_locales['tipo_local'].str.contains('Cafetería', case=False), 'bar', df_locales['tipo_local'])
# Cambiamos los valores que contienen 'Comedy Club' en el nombre del local por 'Club de comedia'
df_locales['tipo_local'] = np.where(df_locales['local'].str.contains('Comedy Club', case=False), 'club de comedia', df_locales['tipo_local'])
# Cambiamos los valores que contienen 'Cine' por 'cine'
df_locales['tipo_local'] = np.where(df_locales['tipo_local'].str.contains('Cine', case=False), 'cine', df_locales['tipo_local'])

df_locales.groupby('tipo_local')['local'].count().reset_index()

,tipo_local,local
0,Escuela de baile,1
1,Espacio de coworking,1
2,Mago,1
3,Sala de conciertos,1
4,bar,11
5,cine,3
6,club de comedia,3
7,hotel,2
8,teatro,25


In [12]:
# Ya con lo que queda definimos las categorías permitidas
categorias_permitidas = ['Cine', 'Club de comedia', 'bar', 'hotel', 'teatro']

# Y cambiamos los valores que no están en las categorías permitidas por 'espacios variados'
df_locales['tipo_local'] = np.where(df_locales['tipo_local'].str.contains('|'.join(categorias_permitidas), case=False, na=False), df_locales['tipo_local'], 'espacios variados')
df_locales.groupby('tipo_local')['local'].count().reset_index()

,tipo_local,local
0,bar,11
1,cine,3
2,club de comedia,3
3,espacios variados,4
4,hotel,2
5,teatro,25


In [13]:
# Hacemos una comprobación

df_locales.isnull().sum()/df_locales.shape[0]

local                         0.000000
direccion                     0.000000
tipo_local                    0.000000
valoracion                    0.000000
numero_reseñas                0.041667
enlace_web                    0.083333
puntuacion_entradas_com       0.333333
numero_entradas_com           0.333333
puntuacion_facebook           0.687500
numero_puntuacion_facebook    0.687500
dtype: float64

# Limpiamos las valoraciones

In [14]:
# Limpiamos las valoraciones para que no nos de problemas, quitamos la coma por punto
df_locales['valoracion'] = df_locales['valoracion'].str.replace(',', '.')
df_locales.head()

,local,direccion,tipo_local,valoracion,numero_reseñas,enlace_web,puntuacion_entradas_com,numero_entradas_com,puntuacion_facebook,numero_puntuacion_facebook
0,Teatro Príncipe Gran Vía,"C. de las Tres Cruces, 8, 28013 Madrid",teatro,4.2,2.601 reseñas de Google,http://www.teatroprincipegranvia.es/,"3,7/5",823 reseñas,"4,3/5",89 votos
1,El Golfo Comedy Club,"C. de las Huertas, 57, 28014 Madrid",club de comedia,4.7,1.082 reseñas de Google,https://elgolfocomedy.com/,"8,9/10",· 1.660 votos,NaN,NaN
2,Teatro Arlequín Gran Vía,"C. de San Bernardo, 5, 28013 Madrid",teatro,4.2,5.264 reseñas de Google,http://teatroarlequingranvia.com/,"3,7/5",329 reseñas,"4,6/5",103 votos
3,Teatro Capitol Gran Vía,"C/ Gran Vía, 41, 28013 Madrid",teatro,4.2,4.902 reseñas de Google,http://www.capitolgranvia.com/,4/5,· 577 reseñas,NaN,NaN
4,Estación Malasaña,"C. del Pez, 16, 28004 Madrid",bar,4.5,344 reseñas de Google,https://www.estacionmalasana.es/,"3,9/5",· 23 votos,NaN,NaN


# Limpiamos el número de reseñas de google

In [15]:
# Reemplazamos 'reseñas de Google' por una cadena vacía en la columna 'numero_reseñas' y quitamos los puntos
df_locales['numero_reseñas'] = df_locales['numero_reseñas'].str.replace('reseñas de Google', '').str.replace('.', '')
df_locales.head()

,local,direccion,tipo_local,valoracion,numero_reseñas,enlace_web,puntuacion_entradas_com,numero_entradas_com,puntuacion_facebook,numero_puntuacion_facebook
0,Teatro Príncipe Gran Vía,"C. de las Tres Cruces, 8, 28013 Madrid",teatro,4.2,2601,http://www.teatroprincipegranvia.es/,"3,7/5",823 reseñas,"4,3/5",89 votos
1,El Golfo Comedy Club,"C. de las Huertas, 57, 28014 Madrid",club de comedia,4.7,1082,https://elgolfocomedy.com/,"8,9/10",· 1.660 votos,NaN,NaN
2,Teatro Arlequín Gran Vía,"C. de San Bernardo, 5, 28013 Madrid",teatro,4.2,5264,http://teatroarlequingranvia.com/,"3,7/5",329 reseñas,"4,6/5",103 votos
3,Teatro Capitol Gran Vía,"C/ Gran Vía, 41, 28013 Madrid",teatro,4.2,4902,http://www.capitolgranvia.com/,4/5,· 577 reseñas,NaN,NaN
4,Estación Malasaña,"C. del Pez, 16, 28004 Madrid",bar,4.5,344,https://www.estacionmalasana.es/,"3,9/5",· 23 votos,NaN,NaN


In [16]:
# Hacemos una comprobación

df_locales.isnull().sum()/df_locales.shape[0]

local                         0.000000
direccion                     0.000000
tipo_local                    0.000000
valoracion                    0.000000
numero_reseñas                0.041667
enlace_web                    0.083333
puntuacion_entradas_com       0.333333
numero_entradas_com           0.333333
puntuacion_facebook           0.687500
numero_puntuacion_facebook    0.687500
dtype: float64

In [17]:
df_locales.loc[df_locales['numero_reseñas'].isnull()]

,local,direccion,tipo_local,valoracion,numero_reseñas,enlace_web,puntuacion_entradas_com,numero_entradas_com,puntuacion_facebook,numero_puntuacion_facebook
33,Bala Perdida Club (Hotel Axel Madrid),"C. de Atocha, 49, 28012 Madrid",hotel,4.4,NaN,https://www.axelhotels.com/int/axel-hotel-madr...,NaN,NaN,NaN,NaN
48,Sleep'n Atocha,"Calle del Dr. Drumen, 4, 28012 Madrid",hotel,4.4,NaN,http://www.sleepnatocha.com/,NaN,NaN,NaN,NaN


In [18]:
# Asignamos manualmente los valores
df_locales.loc[33, 'numero_reseñas'] = 1872
df_locales.loc[48, 'numero_reseñas'] = 1947

# Verificamos los cambios

df_locales.loc[df_locales['numero_reseñas'].isnull()]

,local,direccion,tipo_local,valoracion,numero_reseñas,enlace_web,puntuacion_entradas_com,numero_entradas_com,puntuacion_facebook,numero_puntuacion_facebook


# Limpiamos puntuaciones de entradas.com

Observamos que parte de las puntuaciones recogidas como 'puntuacion_entradas_com' realmente son las de atrapalo, dato que ya tenemos en otro CSV. Las puntuaciones de Atrapalo son las que están sobre 10.

In [19]:
# Convertimos en nulos los valores que contengan '/10' en 'puntuacion_entradas_com'.
mask = df_locales['puntuacion_entradas_com'].notna() & df_locales['puntuacion_entradas_com'].str.contains('/10')
df_locales.loc[mask, 'puntuacion_entradas_com'] = np.nan
df_locales

,local,direccion,tipo_local,valoracion,numero_reseñas,enlace_web,puntuacion_entradas_com,numero_entradas_com,puntuacion_facebook,numero_puntuacion_facebook
0,Teatro Príncipe Gran Vía,"C. de las Tres Cruces, 8, 28013 Madrid",teatro,4.2,2601,http://www.teatroprincipegranvia.es/,"3,7/5",823 reseñas,"4,3/5",89 votos
1,El Golfo Comedy Club,"C. de las Huertas, 57, 28014 Madrid",club de comedia,4.7,1082,https://elgolfocomedy.com/,NaN,· 1.660 votos,NaN,NaN
2,Teatro Arlequín Gran Vía,"C. de San Bernardo, 5, 28013 Madrid",teatro,4.2,5264,http://teatroarlequingranvia.com/,"3,7/5",329 reseñas,"4,6/5",103 votos
3,Teatro Capitol Gran Vía,"C/ Gran Vía, 41, 28013 Madrid",teatro,4.2,4902,http://www.capitolgranvia.com/,4/5,· 577 reseñas,NaN,NaN
4,Estación Malasaña,"C. del Pez, 16, 28004 Madrid",bar,4.5,344,https://www.estacionmalasana.es/,"3,9/5",· 23 votos,NaN,NaN
5,Teatro Alcázar,"C. de Alcalá, 20, 28014 Madrid",teatro,4.3,6730,https://gruposmedia.com/teatro-alcazar/,"3,9/5",· 1.554 reseñas,NaN,NaN
6,Cinesa Fuencarral,"Calle de Fuencarral, 136, 28010 Madrid",cine,4.2,6918,http://www.cinesa.es/Cines/Proyecciones,NaN,NaN,NaN,NaN
7,El Sombrerero Comedy Club,"C. de Trafalgar, 15, 28010 Madrid",club de comedia,4.5,4,NaN,NaN,NaN,NaN,NaN
8,Teatro de las Aguas,"C. de las Aguas, 8, 28005 Madrid",teatro,4.2,1848,http://www.teatrodelasaguas.com/,NaN,902 votos,"4,7/5",116 votos
9,Abonavida,"C. de las Navas de Tolosa, 3, 28013 Madrid",bar,4.3,1274,http://www.abonavida.com/,4/5,103 votos,"4,7/5",3 reseñas


In [20]:
# También reemplazamos comas por puntos para que identifique bien los decimales, asi como el '/5' para evitar confusiones
df_locales['puntuacion_entradas_com'] = df_locales['puntuacion_entradas_com'].str.replace(',', '.').str.replace('/5', '')
df_locales.head()

,local,direccion,tipo_local,valoracion,numero_reseñas,enlace_web,puntuacion_entradas_com,numero_entradas_com,puntuacion_facebook,numero_puntuacion_facebook
0,Teatro Príncipe Gran Vía,"C. de las Tres Cruces, 8, 28013 Madrid",teatro,4.2,2601,http://www.teatroprincipegranvia.es/,3.7,823 reseñas,"4,3/5",89 votos
1,El Golfo Comedy Club,"C. de las Huertas, 57, 28014 Madrid",club de comedia,4.7,1082,https://elgolfocomedy.com/,NaN,· 1.660 votos,NaN,NaN
2,Teatro Arlequín Gran Vía,"C. de San Bernardo, 5, 28013 Madrid",teatro,4.2,5264,http://teatroarlequingranvia.com/,3.7,329 reseñas,"4,6/5",103 votos
3,Teatro Capitol Gran Vía,"C/ Gran Vía, 41, 28013 Madrid",teatro,4.2,4902,http://www.capitolgranvia.com/,4,· 577 reseñas,NaN,NaN
4,Estación Malasaña,"C. del Pez, 16, 28004 Madrid",bar,4.5,344,https://www.estacionmalasana.es/,3.9,· 23 votos,NaN,NaN


In [21]:
# Hacemos una comprobación

df_locales.isnull().sum()/df_locales.shape[0]

local                         0.000000
direccion                     0.000000
tipo_local                    0.000000
valoracion                    0.000000
numero_reseñas                0.000000
enlace_web                    0.083333
puntuacion_entradas_com       0.541667
numero_entradas_com           0.333333
puntuacion_facebook           0.687500
numero_puntuacion_facebook    0.687500
dtype: float64

# Limpiamos el número de reseñas de entradas.com

Aqui también tenemos valores de atrapalo que no corresponden, asi que los volvemos nulos

In [22]:
# Convertimos en nulos los valores de 'numero_entradas_com' donde 'puntuacion_entradas_com' es nulo
df_locales.loc[df_locales['puntuacion_entradas_com'].isnull(), 'numero_entradas_com'] = np.nan
df_locales.head()

,local,direccion,tipo_local,valoracion,numero_reseñas,enlace_web,puntuacion_entradas_com,numero_entradas_com,puntuacion_facebook,numero_puntuacion_facebook
0,Teatro Príncipe Gran Vía,"C. de las Tres Cruces, 8, 28013 Madrid",teatro,4.2,2601,http://www.teatroprincipegranvia.es/,3.7,823 reseñas,"4,3/5",89 votos
1,El Golfo Comedy Club,"C. de las Huertas, 57, 28014 Madrid",club de comedia,4.7,1082,https://elgolfocomedy.com/,NaN,NaN,NaN,NaN
2,Teatro Arlequín Gran Vía,"C. de San Bernardo, 5, 28013 Madrid",teatro,4.2,5264,http://teatroarlequingranvia.com/,3.7,329 reseñas,"4,6/5",103 votos
3,Teatro Capitol Gran Vía,"C/ Gran Vía, 41, 28013 Madrid",teatro,4.2,4902,http://www.capitolgranvia.com/,4,· 577 reseñas,NaN,NaN
4,Estación Malasaña,"C. del Pez, 16, 28004 Madrid",bar,4.5,344,https://www.estacionmalasana.es/,3.9,· 23 votos,NaN,NaN


In [23]:
# Quitamos el texto
df_locales['numero_entradas_com'] = df_locales['numero_entradas_com'].str.extract(r'(\d+\.?\d*)').astype(float)
df_locales.head(2)

,local,direccion,tipo_local,valoracion,numero_reseñas,enlace_web,puntuacion_entradas_com,numero_entradas_com,puntuacion_facebook,numero_puntuacion_facebook
0,Teatro Príncipe Gran Vía,"C. de las Tres Cruces, 8, 28013 Madrid",teatro,4.2,2601,http://www.teatroprincipegranvia.es/,3.7,823.0,"4,3/5",89 votos
1,El Golfo Comedy Club,"C. de las Huertas, 57, 28014 Madrid",club de comedia,4.7,1082,https://elgolfocomedy.com/,NaN,NaN,NaN,NaN


In [24]:
# Hacemos una comprobación

df_locales.isnull().sum()/df_locales.shape[0]

local                         0.000000
direccion                     0.000000
tipo_local                    0.000000
valoracion                    0.000000
numero_reseñas                0.000000
enlace_web                    0.083333
puntuacion_entradas_com       0.541667
numero_entradas_com           0.541667
puntuacion_facebook           0.687500
numero_puntuacion_facebook    0.687500
dtype: float64

# Limpiamos puntuaciones de facebook

In [25]:
# Convertimos en nulos los valores que contengan '/10' en 'puntuacion_facebook'.
mask = df_locales['puntuacion_facebook'].notna() & df_locales['puntuacion_facebook'].str.contains('/10')
df_locales.loc[mask, 'puntuacion_facebook'] = np.nan
df_locales

,local,direccion,tipo_local,valoracion,numero_reseñas,enlace_web,puntuacion_entradas_com,numero_entradas_com,puntuacion_facebook,numero_puntuacion_facebook
0,Teatro Príncipe Gran Vía,"C. de las Tres Cruces, 8, 28013 Madrid",teatro,4.2,2601,http://www.teatroprincipegranvia.es/,3.7,823.000,"4,3/5",89 votos
1,El Golfo Comedy Club,"C. de las Huertas, 57, 28014 Madrid",club de comedia,4.7,1082,https://elgolfocomedy.com/,NaN,NaN,NaN,NaN
2,Teatro Arlequín Gran Vía,"C. de San Bernardo, 5, 28013 Madrid",teatro,4.2,5264,http://teatroarlequingranvia.com/,3.7,329.000,"4,6/5",103 votos
3,Teatro Capitol Gran Vía,"C/ Gran Vía, 41, 28013 Madrid",teatro,4.2,4902,http://www.capitolgranvia.com/,4,577.000,NaN,NaN
4,Estación Malasaña,"C. del Pez, 16, 28004 Madrid",bar,4.5,344,https://www.estacionmalasana.es/,3.9,23.000,NaN,NaN
5,Teatro Alcázar,"C. de Alcalá, 20, 28014 Madrid",teatro,4.3,6730,https://gruposmedia.com/teatro-alcazar/,3.9,1.554,NaN,NaN
6,Cinesa Fuencarral,"Calle de Fuencarral, 136, 28010 Madrid",cine,4.2,6918,http://www.cinesa.es/Cines/Proyecciones,NaN,NaN,NaN,NaN
7,El Sombrerero Comedy Club,"C. de Trafalgar, 15, 28010 Madrid",club de comedia,4.5,4,NaN,NaN,NaN,NaN,NaN
8,Teatro de las Aguas,"C. de las Aguas, 8, 28005 Madrid",teatro,4.2,1848,http://www.teatrodelasaguas.com/,NaN,NaN,"4,7/5",116 votos
9,Abonavida,"C. de las Navas de Tolosa, 3, 28013 Madrid",bar,4.3,1274,http://www.abonavida.com/,4,103.000,"4,7/5",3 reseñas


In [26]:
# Reemplazamos comas por puntos para que identifique bien los decimales, asi como el '/5' para evitar confusiones
df_locales['puntuacion_facebook'] = df_locales['puntuacion_facebook'].str.replace(',', '.').str.replace('/5', '')
df_locales

,local,direccion,tipo_local,valoracion,numero_reseñas,enlace_web,puntuacion_entradas_com,numero_entradas_com,puntuacion_facebook,numero_puntuacion_facebook
0,Teatro Príncipe Gran Vía,"C. de las Tres Cruces, 8, 28013 Madrid",teatro,4.2,2601,http://www.teatroprincipegranvia.es/,3.7,823.000,4.3,89 votos
1,El Golfo Comedy Club,"C. de las Huertas, 57, 28014 Madrid",club de comedia,4.7,1082,https://elgolfocomedy.com/,NaN,NaN,NaN,NaN
2,Teatro Arlequín Gran Vía,"C. de San Bernardo, 5, 28013 Madrid",teatro,4.2,5264,http://teatroarlequingranvia.com/,3.7,329.000,4.6,103 votos
3,Teatro Capitol Gran Vía,"C/ Gran Vía, 41, 28013 Madrid",teatro,4.2,4902,http://www.capitolgranvia.com/,4,577.000,NaN,NaN
4,Estación Malasaña,"C. del Pez, 16, 28004 Madrid",bar,4.5,344,https://www.estacionmalasana.es/,3.9,23.000,NaN,NaN
5,Teatro Alcázar,"C. de Alcalá, 20, 28014 Madrid",teatro,4.3,6730,https://gruposmedia.com/teatro-alcazar/,3.9,1.554,NaN,NaN
6,Cinesa Fuencarral,"Calle de Fuencarral, 136, 28010 Madrid",cine,4.2,6918,http://www.cinesa.es/Cines/Proyecciones,NaN,NaN,NaN,NaN
7,El Sombrerero Comedy Club,"C. de Trafalgar, 15, 28010 Madrid",club de comedia,4.5,4,NaN,NaN,NaN,NaN,NaN
8,Teatro de las Aguas,"C. de las Aguas, 8, 28005 Madrid",teatro,4.2,1848,http://www.teatrodelasaguas.com/,NaN,NaN,4.7,116 votos
9,Abonavida,"C. de las Navas de Tolosa, 3, 28013 Madrid",bar,4.3,1274,http://www.abonavida.com/,4,103.000,4.7,3 reseñas


In [27]:
# Hacemos una comprobación

df_locales.isnull().sum()/df_locales.shape[0]

local                         0.000000
direccion                     0.000000
tipo_local                    0.000000
valoracion                    0.000000
numero_reseñas                0.000000
enlace_web                    0.083333
puntuacion_entradas_com       0.541667
numero_entradas_com           0.541667
puntuacion_facebook           0.708333
numero_puntuacion_facebook    0.687500
dtype: float64

# Limpiamos el número de puntuaciones de facebook

In [28]:
# Convertimos en nulos los valores de 'numero_puntuacion_facebook' donde 'puntuacion_facebook' es nulo
df_locales.loc[df_locales['puntuacion_facebook'].isnull(), 'numero_puntuacion_facebook'] = np.nan
df_locales.head()

,local,direccion,tipo_local,valoracion,numero_reseñas,enlace_web,puntuacion_entradas_com,numero_entradas_com,puntuacion_facebook,numero_puntuacion_facebook
0,Teatro Príncipe Gran Vía,"C. de las Tres Cruces, 8, 28013 Madrid",teatro,4.2,2601,http://www.teatroprincipegranvia.es/,3.7,823.0,4.3,89 votos
1,El Golfo Comedy Club,"C. de las Huertas, 57, 28014 Madrid",club de comedia,4.7,1082,https://elgolfocomedy.com/,NaN,NaN,NaN,NaN
2,Teatro Arlequín Gran Vía,"C. de San Bernardo, 5, 28013 Madrid",teatro,4.2,5264,http://teatroarlequingranvia.com/,3.7,329.0,4.6,103 votos
3,Teatro Capitol Gran Vía,"C/ Gran Vía, 41, 28013 Madrid",teatro,4.2,4902,http://www.capitolgranvia.com/,4,577.0,NaN,NaN
4,Estación Malasaña,"C. del Pez, 16, 28004 Madrid",bar,4.5,344,https://www.estacionmalasana.es/,3.9,23.0,NaN,NaN


In [29]:
# Convertimos en nulos los valores de 'numero_entradas_com' donde 'puntuacion_entradas_com' es nulo
def extract_numbers(text):
    numbers = re.findall(r'\d+', str(text))
    return int(numbers[0]) if numbers else None

df_locales['numero_puntuacion_facebook'] = df_locales['numero_puntuacion_facebook'].apply(extract_numbers)
df_locales.sample(5)

,local,direccion,tipo_local,valoracion,numero_reseñas,enlace_web,puntuacion_entradas_com,numero_entradas_com,puntuacion_facebook,numero_puntuacion_facebook
35,Sala Kathedral,"Corre. Baja de San Pablo, 39-1, 28004 Madrid",bar,3.7,158,https://instagram.com/tocacabsnamadrid,NaN,NaN,NaN,NaN
0,Teatro Príncipe Gran Vía,"C. de las Tres Cruces, 8, 28013 Madrid",teatro,4.2,2601,http://www.teatroprincipegranvia.es/,3.7,823.00,4.3,89.0
33,Bala Perdida Club (Hotel Axel Madrid),"C. de Atocha, 49, 28012 Madrid",hotel,4.4,1872,https://www.axelhotels.com/int/axel-hotel-madr...,NaN,NaN,NaN,NaN
11,Teatros Luchana,"C. de Luchana, 38, 28010 Madrid",teatro,4.5,3371,https://teatrosluchana.es/,4.6,232.00,4.5,215.0
32,Sala Galileo Galilei,"C. de Galileo, 100, 28015 Madrid",espacios variados,4.3,4057,http://www.salagalileogalilei.com/,3.7,1.67,NaN,NaN


In [30]:
# Quitamos el texto
df_locales['numero_puntuacion_facebook'] = df_locales['numero_puntuacion_facebook'].replace(0, np.nan)

df_locales.head(2)

,local,direccion,tipo_local,valoracion,numero_reseñas,enlace_web,puntuacion_entradas_com,numero_entradas_com,puntuacion_facebook,numero_puntuacion_facebook
0,Teatro Príncipe Gran Vía,"C. de las Tres Cruces, 8, 28013 Madrid",teatro,4.2,2601,http://www.teatroprincipegranvia.es/,3.7,823.0,4.3,89.0
1,El Golfo Comedy Club,"C. de las Huertas, 57, 28014 Madrid",club de comedia,4.7,1082,https://elgolfocomedy.com/,NaN,NaN,NaN,NaN


In [31]:
# Hacemos una comprobación

df_locales.isnull().sum()/df_locales.shape[0]

local                         0.000000
direccion                     0.000000
tipo_local                    0.000000
valoracion                    0.000000
numero_reseñas                0.000000
enlace_web                    0.083333
puntuacion_entradas_com       0.541667
numero_entradas_com           0.541667
puntuacion_facebook           0.708333
numero_puntuacion_facebook    0.708333
dtype: float64

# Limpiamos los locales

In [32]:
# Primero vemos que tenemos
lista_locales = df_locales['local'].unique()
sorted(lista_locales)

['Abonavida',
 'Bala Perdida Club (Hotel Axel Madrid)',
 'Beer Station - Sala 1',
 'Cafetería Valverde',
 'Cervecería Valverde Comedy Club - Gran Vía',
 'Cine Palacio de la Prensa - Sala 1',
 'Cine Palacio de la Prensa - Sala 3',
 'Cinesa Fuencarral',
 'Clandestino Café Teatro',
 'Collage Burlesque',
 'El Golfo Comedy Club',
 'El Sombrerero Comedy Club',
 'Escuela de Danza y Centro Deportivo Belo Art',
 'Estación Malasaña',
 'Estupenda Bar',
 'Gran teatro Caixabank Príncipe Pío',
 'Intruso Bar',
 'La Casa de Rovodorovsky Teatro',
 'La Cripta Mágica',
 'La Nota Rock',
 'La chocita del Loro - Carabanchel',
 'Mad Beer Hall',
 'OFF LATINA',
 'Pequeño Teatro Gran Vía de Madrid',
 'Platea Café Teatro',
 'Sala Galileo Galilei',
 'Sala Houdini',
 'Sala Kathedral',
 "Sleep'n Atocha",
 'Teatro Alcázar',
 'Teatro Amaya',
 'Teatro Arlequín Gran Vía',
 'Teatro Bellas Artes',
 'Teatro Capitol Gran Vía',
 'Teatro Chocita del Loro - Avenida Brasil',
 'Teatro Chocita del Loro - Gran Vía',
 'Teatro EDP 

Quitamos los números de sala y todo lo que sobra detras de los guiones excepto en La chocita del Loro, ya que es lo que diferencia una sala de otra

In [33]:
# Primero ponemos a todas las chocitas igual
df_locales['local'] = df_locales['local'].replace('La chocita del Loro - Carabanchel', 'Teatro Chocita del Loro - Carabanchel')

In [34]:
# Y ahora creamos una nueva columna para hacer la limpieza de las salas 
df_locales['local_modificada'] = df_locales['local'].apply(lambda x: x if 'Chocita del Loro' in x else x.split(' - ')[0].strip())
df_locales.head()

,local,direccion,tipo_local,valoracion,numero_reseñas,enlace_web,puntuacion_entradas_com,numero_entradas_com,puntuacion_facebook,numero_puntuacion_facebook,local_modificada
0,Teatro Príncipe Gran Vía,"C. de las Tres Cruces, 8, 28013 Madrid",teatro,4.2,2601,http://www.teatroprincipegranvia.es/,3.7,823.0,4.3,89.0,Teatro Príncipe Gran Vía
1,El Golfo Comedy Club,"C. de las Huertas, 57, 28014 Madrid",club de comedia,4.7,1082,https://elgolfocomedy.com/,NaN,NaN,NaN,NaN,El Golfo Comedy Club
2,Teatro Arlequín Gran Vía,"C. de San Bernardo, 5, 28013 Madrid",teatro,4.2,5264,http://teatroarlequingranvia.com/,3.7,329.0,4.6,103.0,Teatro Arlequín Gran Vía
3,Teatro Capitol Gran Vía,"C/ Gran Vía, 41, 28013 Madrid",teatro,4.2,4902,http://www.capitolgranvia.com/,4,577.0,NaN,NaN,Teatro Capitol Gran Vía
4,Estación Malasaña,"C. del Pez, 16, 28004 Madrid",bar,4.5,344,https://www.estacionmalasana.es/,3.9,23.0,NaN,NaN,Estación Malasaña


In [35]:
# Ahora sustituimos los valores y eliminamos la columna que nos sobra
df_locales['local'] = df_locales['local_modificada']
df_locales = df_locales.drop('local_modificada', axis=1)
df_locales

,local,direccion,tipo_local,valoracion,numero_reseñas,enlace_web,puntuacion_entradas_com,numero_entradas_com,puntuacion_facebook,numero_puntuacion_facebook
0,Teatro Príncipe Gran Vía,"C. de las Tres Cruces, 8, 28013 Madrid",teatro,4.2,2601,http://www.teatroprincipegranvia.es/,3.7,823.000,4.3,89.0
1,El Golfo Comedy Club,"C. de las Huertas, 57, 28014 Madrid",club de comedia,4.7,1082,https://elgolfocomedy.com/,NaN,NaN,NaN,NaN
2,Teatro Arlequín Gran Vía,"C. de San Bernardo, 5, 28013 Madrid",teatro,4.2,5264,http://teatroarlequingranvia.com/,3.7,329.000,4.6,103.0
3,Teatro Capitol Gran Vía,"C/ Gran Vía, 41, 28013 Madrid",teatro,4.2,4902,http://www.capitolgranvia.com/,4,577.000,NaN,NaN
4,Estación Malasaña,"C. del Pez, 16, 28004 Madrid",bar,4.5,344,https://www.estacionmalasana.es/,3.9,23.000,NaN,NaN
5,Teatro Alcázar,"C. de Alcalá, 20, 28014 Madrid",teatro,4.3,6730,https://gruposmedia.com/teatro-alcazar/,3.9,1.554,NaN,NaN
6,Cinesa Fuencarral,"Calle de Fuencarral, 136, 28010 Madrid",cine,4.2,6918,http://www.cinesa.es/Cines/Proyecciones,NaN,NaN,NaN,NaN
7,El Sombrerero Comedy Club,"C. de Trafalgar, 15, 28010 Madrid",club de comedia,4.5,4,NaN,NaN,NaN,NaN,NaN
8,Teatro de las Aguas,"C. de las Aguas, 8, 28005 Madrid",teatro,4.2,1848,http://www.teatrodelasaguas.com/,NaN,NaN,4.7,116.0
9,Abonavida,"C. de las Navas de Tolosa, 3, 28013 Madrid",bar,4.3,1274,http://www.abonavida.com/,4,103.000,4.7,3.0


# Comprobamos y corregimos los tipos de dato

In [36]:
df_locales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48 entries, 0 to 51
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   local                       48 non-null     object 
 1   direccion                   48 non-null     object 
 2   tipo_local                  48 non-null     object 
 3   valoracion                  48 non-null     object 
 4   numero_reseñas              48 non-null     object 
 5   enlace_web                  44 non-null     object 
 6   puntuacion_entradas_com     22 non-null     object 
 7   numero_entradas_com         22 non-null     float64
 8   puntuacion_facebook         14 non-null     object 
 9   numero_puntuacion_facebook  14 non-null     float64
dtypes: float64(2), object(8)
memory usage: 5.2+ KB


In [37]:
# Hacemos una comprobación

df_locales.isnull().sum()/df_locales.shape[0]

local                         0.000000
direccion                     0.000000
tipo_local                    0.000000
valoracion                    0.000000
numero_reseñas                0.000000
enlace_web                    0.083333
puntuacion_entradas_com       0.541667
numero_entradas_com           0.541667
puntuacion_facebook           0.708333
numero_puntuacion_facebook    0.708333
dtype: float64

In [38]:
#df_locales['numero_reseñas'] = df_locales['numero_reseñas'].fillna(0)
#df_locales['puntuacion_facebook'] = df_locales['puntuacion_facebook'].fillna(0)

In [39]:
# Necesitamos que el precio y la duración tengan formatos númericos

df_locales['valoracion'] = pd.to_numeric(df_locales['valoracion'], errors='coerce')
df_locales['numero_reseñas'] = pd.to_numeric(df_locales['numero_reseñas'], errors='coerce')
df_locales['puntuacion_facebook'] = pd.to_numeric(df_locales['puntuacion_facebook'], errors='coerce')
df_locales['puntuacion_entradas_com'] = pd.to_numeric(df_locales['puntuacion_entradas_com'], errors='ignore')

df_locales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48 entries, 0 to 51
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   local                       48 non-null     object 
 1   direccion                   48 non-null     object 
 2   tipo_local                  48 non-null     object 
 3   valoracion                  48 non-null     float64
 4   numero_reseñas              48 non-null     int64  
 5   enlace_web                  44 non-null     object 
 6   puntuacion_entradas_com     22 non-null     float64
 7   numero_entradas_com         22 non-null     float64
 8   puntuacion_facebook         14 non-null     float64
 9   numero_puntuacion_facebook  14 non-null     float64
dtypes: float64(5), int64(1), object(4)
memory usage: 5.2+ KB


In [40]:
# Hacemos una comprobación

df_locales.isnull().sum()/df_locales.shape[0]

local                         0.000000
direccion                     0.000000
tipo_local                    0.000000
valoracion                    0.000000
numero_reseñas                0.000000
enlace_web                    0.083333
puntuacion_entradas_com       0.541667
numero_entradas_com           0.541667
puntuacion_facebook           0.708333
numero_puntuacion_facebook    0.708333
dtype: float64

In [41]:
df_locales.head()

,local,direccion,tipo_local,valoracion,numero_reseñas,enlace_web,puntuacion_entradas_com,numero_entradas_com,puntuacion_facebook,numero_puntuacion_facebook
0,Teatro Príncipe Gran Vía,"C. de las Tres Cruces, 8, 28013 Madrid",teatro,4.2,2601,http://www.teatroprincipegranvia.es/,3.7,823.0,4.3,89.0
1,El Golfo Comedy Club,"C. de las Huertas, 57, 28014 Madrid",club de comedia,4.7,1082,https://elgolfocomedy.com/,NaN,NaN,NaN,NaN
2,Teatro Arlequín Gran Vía,"C. de San Bernardo, 5, 28013 Madrid",teatro,4.2,5264,http://teatroarlequingranvia.com/,3.7,329.0,4.6,103.0
3,Teatro Capitol Gran Vía,"C/ Gran Vía, 41, 28013 Madrid",teatro,4.2,4902,http://www.capitolgranvia.com/,4.0,577.0,NaN,NaN
4,Estación Malasaña,"C. del Pez, 16, 28004 Madrid",bar,4.5,344,https://www.estacionmalasana.es/,3.9,23.0,NaN,NaN


# Por último, renombramos columnas

In [42]:
df_locales.columns

Index(['local', 'direccion', 'tipo_local', 'valoracion', 'numero_reseñas',
       'enlace_web', 'puntuacion_entradas_com', 'numero_entradas_com',
       'puntuacion_facebook', 'numero_puntuacion_facebook'],
      dtype='object')

In [43]:
df_locales = df_locales.rename(columns={'valoracion': 'valoracion_google',
                                        'numero_reseñas': 'numero_reseñas_google'})
df_locales.head()

,local,direccion,tipo_local,valoracion_google,numero_reseñas_google,enlace_web,puntuacion_entradas_com,numero_entradas_com,puntuacion_facebook,numero_puntuacion_facebook
0,Teatro Príncipe Gran Vía,"C. de las Tres Cruces, 8, 28013 Madrid",teatro,4.2,2601,http://www.teatroprincipegranvia.es/,3.7,823.0,4.3,89.0
1,El Golfo Comedy Club,"C. de las Huertas, 57, 28014 Madrid",club de comedia,4.7,1082,https://elgolfocomedy.com/,NaN,NaN,NaN,NaN
2,Teatro Arlequín Gran Vía,"C. de San Bernardo, 5, 28013 Madrid",teatro,4.2,5264,http://teatroarlequingranvia.com/,3.7,329.0,4.6,103.0
3,Teatro Capitol Gran Vía,"C/ Gran Vía, 41, 28013 Madrid",teatro,4.2,4902,http://www.capitolgranvia.com/,4.0,577.0,NaN,NaN
4,Estación Malasaña,"C. del Pez, 16, 28004 Madrid",bar,4.5,344,https://www.estacionmalasana.es/,3.9,23.0,NaN,NaN


In [44]:
df_locales

,local,direccion,tipo_local,valoracion_google,numero_reseñas_google,enlace_web,puntuacion_entradas_com,numero_entradas_com,puntuacion_facebook,numero_puntuacion_facebook
0,Teatro Príncipe Gran Vía,"C. de las Tres Cruces, 8, 28013 Madrid",teatro,4.2,2601,http://www.teatroprincipegranvia.es/,3.7,823.000,4.3,89.0
1,El Golfo Comedy Club,"C. de las Huertas, 57, 28014 Madrid",club de comedia,4.7,1082,https://elgolfocomedy.com/,NaN,NaN,NaN,NaN
2,Teatro Arlequín Gran Vía,"C. de San Bernardo, 5, 28013 Madrid",teatro,4.2,5264,http://teatroarlequingranvia.com/,3.7,329.000,4.6,103.0
3,Teatro Capitol Gran Vía,"C/ Gran Vía, 41, 28013 Madrid",teatro,4.2,4902,http://www.capitolgranvia.com/,4.0,577.000,NaN,NaN
4,Estación Malasaña,"C. del Pez, 16, 28004 Madrid",bar,4.5,344,https://www.estacionmalasana.es/,3.9,23.000,NaN,NaN
5,Teatro Alcázar,"C. de Alcalá, 20, 28014 Madrid",teatro,4.3,6730,https://gruposmedia.com/teatro-alcazar/,3.9,1.554,NaN,NaN
6,Cinesa Fuencarral,"Calle de Fuencarral, 136, 28010 Madrid",cine,4.2,6918,http://www.cinesa.es/Cines/Proyecciones,NaN,NaN,NaN,NaN
7,El Sombrerero Comedy Club,"C. de Trafalgar, 15, 28010 Madrid",club de comedia,4.5,4,NaN,NaN,NaN,NaN,NaN
8,Teatro de las Aguas,"C. de las Aguas, 8, 28005 Madrid",teatro,4.2,1848,http://www.teatrodelasaguas.com/,NaN,NaN,4.7,116.0
9,Abonavida,"C. de las Navas de Tolosa, 3, 28013 Madrid",bar,4.3,1274,http://www.abonavida.com/,4.0,103.000,4.7,3.0


# Guardado

Pasamos a CSV lo que tenemos.

In [46]:
# Especificamos la ruta y el nombre del archivo CSV
ruta_archivo_csv = '01_data\datos_locales_ampliado_limpio.csv'

# Guardamos el nuevo DataFrame en un archivo CSV
df_locales.to_csv(ruta_archivo_csv, index=False)